# Useful conversion functions


First changes from CVAT yoloV5 style so that you can change the class number. Thus you can add it on top of coco dataset 

In [95]:
import os
import shutil

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
# print(coco_additions_path)
additional_classes = os.listdir(coco_additions_path)
# print(additional_classes)


for obj in additional_classes:
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))

  try: 
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    shutil.copyfile(train_txt_file, os.path.normpath(os.path.join(new_obj_path,"train_original.txt")))
  except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'c:\\Users\\justm\\Documents\\OID_YoloV5\\yolov5\\datasets\\coco-additions\\finished\\train.txt'


In [96]:
# Change input value
import os 

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
# print(coco_additions_path)
additional_classes = os.listdir(coco_additions_path)
# print(additional_classes)


for obj in additional_classes:
  new_obj_path=os.path.join(coco_additions_path,obj)

  try:
    obj_train_data = os.path.join(new_obj_path,"obj_train_data") # get the specific file 
    additional_obj_train_data = [os.path.normpath(os.path.join(obj_train_data,file))  for file in os.listdir(obj_train_data) if  file.endswith(".txt")] # find txt in it
    class_number  = int(input(f"[{obj}]Please enter a the new class number as an integer"))

    for text_file in additional_obj_train_data: 
      annotations = []
      with open(text_file) as f: # open text file
          for line in f: # for each line
            labels = line.split() 
            if (not labels[0].isdigit()):
              raise Exception(text_file, "label is not a digit (possible still a label not a number)") 

            labels[0] = class_number
            newline = str(labels[0]) + " " + str(labels[1]) + " " + str(labels[2]) + " " + str(labels[3]) + " " + str(labels[4])
            line = line.replace(line, newline)
            annotations.append(line) # List of string for each line (replace first value with class number entered)
          f.close()

      with open(text_file, "w") as outfile:
        for line in annotations:
            outfile.write(line)
            outfile.write("\n")
        outfile.close()

  except Exception as e: 
    print(new_obj_path, "Does not have 'obj_train_data' folder in it, skipped")




c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_hoverboard Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_scooter Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_unicycle Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Does not have 'obj_train_data' folder in it, skipped


Change the ```train.txt``` file so that it matches the coco2017 ```train2017.txt``` style

coco train2017.txt style = ```./images/train2017/000000109622.jpg```

cvat yolo style  = ```data/obj_train_data/hoverboard_0.jpg```


In [97]:
import os
import shutil

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
# print(coco_additions_path)
additional_classes = os.listdir(coco_additions_path)
# print(additional_classes)


for obj in additional_classes:
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))

  try: 
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    shutil.copyfile(os.path.normpath(os.path.join(new_obj_path,"train_original.txt")),train_txt_file)

    annotations = []
    with open(train_txt_file) as f: # open text file
        for line in f: # for each line
          image_name = line.split("/")[-1].strip() # removes \n at the end as well
          image_line = f"./images/train2017/{image_name}"
          annotations.append(image_line) # List of string for each line (replace first value with class number entered)
        f.close()
    with open(train_txt_file, "w") as outfile:
      for line in annotations:
          outfile.write(line)
          outfile.write("\n")
      outfile.close()
  except Exception as e: 
    print(new_obj_path, "Does not have 'train.txt' folder in it, skipped")


c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\finished Does not have 'train.txt' folder in it, skipped


# Merge the coco2017.txt and the cvat train.txt

BUGS: allows duplicates of the same, however yoloV5 training model should be able to deal with it

In [98]:
import os
import shutil
coco2017_train_path = os.path.join(os.getcwd(),"datasets/coco2017/train2017.txt") # opens the train.txt path
shutil.copyfile(os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/train2017_original.txt")),coco2017_train_path) # copy original to train 

coco2017_file_obj = open(coco2017_train_path, 'a') # open folder to append stuff

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
additional_classes = os.listdir(coco_additions_path)
for obj in additional_classes:
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))

  try: 
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    with open(train_txt_file) as f: # open text file
          for line in f: # for each line
            coco2017_file_obj.write(line) # append to the end
          f.close() # close file when done
    print(new_obj_path, "Has been appended to datasets/coco2017/train2017.txt")
  except:
    print(new_obj_path, "Does not have 'train.txt' folder in it, skipped")
coco2017_file_obj.close()

c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\bicycle_helmet Has been appended to datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\electric_hoverboard Has been appended to datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\electric_scooter Has been appended to datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\electric_unicycle Has been appended to datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\finished Does not have 'train.txt' folder in it, skipped


# Move images to coco dataset

In [99]:
import os 

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
# print(coco_additions_path)
additional_classes = os.listdir(coco_additions_path)
# print(additional_classes)


for obj in additional_classes:
  new_obj_path=os.path.join(coco_additions_path,obj)
  try:
    obj_train_data = os.path.join(new_obj_path,"obj_train_data") # get the specific file 
    jpg_names = [os.path.splitext(file)[0]  for file in os.listdir(obj_train_data) if  file.endswith(".jpg")]
    # C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
    coco_image_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/images/train2017"))
    coco_label_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/labels/train2017"))

    
    for image in jpg_names:
      # shutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")
      label_file_name = image + ".jpg"
      jpg_file_name = image + ".txt"
      # print(os.path.normpath(os.path.join(obj_train_data,label_file_name)),os.path.normpath(os.path.join(coco_image_path,label_file_name)))
      shutil.move(os.path.normpath(os.path.join(obj_train_data,label_file_name)),os.path.normpath(os.path.join(coco_image_path,label_file_name)))
      shutil.move(os.path.normpath(os.path.join(obj_train_data,jpg_file_name)),os.path.normpath(os.path.join(coco_label_path,jpg_file_name)))

    print(f"[Moved] {obj} images and labels to coco paths {coco_image_path}")
  except Exception as e: 
    print(new_obj_path, "Does not have 'obj_train_data' folder in it, skipped")




[Moved] bicycle_helmet images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_hoverboard images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_scooter images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_unicycle images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Does not have 'obj_train_data' folder in it, skipped


# Converting OIDV4 to CVAT format

Under work

TODO: 
- move files to obj_train_data
- train_original.txt (```data/obj_train_data/escooter_0.jpg```) - format
- create train.txt, obj.data, obj.names



In [ ]:
# Function to create train.txt from obj_train_data folder

object_name = "bicycle_helmet"

obj_train_path = os.path.join(os.getcwd(),"datasets/coco-additions/"+ object_name + "/obj_train_data")

print(obj_train_path)
train_files = os.listdir(obj_train_path)
# print(train_files)

annotations= []
for file_names in [file for file in train_files if file.endswith(".jpg")]: # for each line
  image_line = f"data/obj_train_data/{file_names}"
  annotations.append(image_line) # List of string for each line (replace first value with class number entered)


# for all images rename to <class_name>_number
for idx,files_names in enumerate(annotations):
  # obj_train_path # folder path
  txt_name = os.path.splitext(os.path.basename(files_names))[0]  + ".txt"
  jpg_name = os.path.basename(files_names)
  
  # print(obj_train_path + f"/{txt_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".txt" )
  # print(obj_train_path + f"/{jpg_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".jpg")
  try: 
    os.rename(obj_train_path + f"/{txt_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".txt" )
    os.rename(obj_train_path + f"/{jpg_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".jpg") 
  except Exception as e:
    print(f'File {txt_name} missing')

print(os.path.join(os.getcwd(),"datasets/coco-additions/"+ object_name + "/train_original.txt"))
with open(os.path.join(os.getcwd(),"datasets/coco-additions/"+ object_name + "/train_original.txt"), 'w') as outfile:
  for line in annotations:
      outfile.write(line)
      outfile.write("\n")
outfile.close()

shutil.copy(os.path.join(os.getcwd(),"datasets/coco-additions/"+ object_name + "/train_original.txt"), os.path.join(os.getcwd(),"datasets/coco-additions/"+ object_name + "/train.txt"))

# Selecting a custom amount of training samples

In [101]:
# train2017.txt
# Standard format     ./images/train2017/000000109622.jpg     
limit = 1000000 # only removes from coco dataset

coco_train_txt = os.path.join(os.getcwd(),"datasets/coco2017/train2017.txt")
print(coco_train_txt)

keep_lines = []
with open(coco_train_txt) as f: # open text file
    for line in f: # for each line
      file_name = os.path.splitext(os.path.basename(line))[0]
      if file_name.isdigit(): # is a digit number means its from coco 
        if len(keep_lines) < limit:
          keep_lines.append(file_name)
      else:
        keep_lines.append(file_name)



with open(os.path.join(os.getcwd(),"datasets/coco2017/" + f"train{limit}.txt"), 'w') as outfile:
  for file_name in keep_lines:
      line_value = f"./images/train2017/{file_name}.jpg"
      outfile.write(line_value)
      outfile.write("\n")
outfile.close()


print(new_obj_path, f"Has been appended to train{limit}.txt")

c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Has been appended to train1000000.txt


# Delete all images that dont start with 0000 
Thus not part of original coco

In [92]:
# C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017

images_path = r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017"
label_path = r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\labels\train2017"


images_files = os.listdir(images_path)

for names in images_files: 
  if names[0:4] != "0000": # original coco images are 0000
    jpg_path = os.path.join(images_path,names)
    label_names = names.split(".")[0] + ".txt"
    txt_path = os.path.join(label_path,label_names)
    os.remove(jpg_path)
    os.remove(txt_path)


# Combining Jacks Yolo


In [29]:
from string import punctuation
any(p in "my_text" for p in punctuation)

True

In [32]:
import os 
import shutil
jack_image_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\images\train")
jack_label_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train")

numbers = [str(i) for i in range(0,10)]

print(jack_image_path)
train_files = os.listdir(jack_image_path)
# print(train_files)

annotations= []
for file_names in [file for file in train_files if file.endswith(".jpg")]: # for each line
  f_name=os.path.splitext(file_names)[0]
  if not any(p in f_name for p in punctuation):
    image_line = f"./images/train2017/{file_names}"
    annotations.append(image_line) # List of string for each line (replace first value with class number entered)



print(os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt"))
with open(os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt"), 'w') as outfile:
  for line in annotations:
      outfile.write(line)
      outfile.write("\n")
outfile.close()


C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\images\train
C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt


In [53]:
# Adjust head label to be 84
obj = 'head'
current_class_number = 85
new_class_number = 84
jack_label_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train")
print(jack_label_path)

for file_name in os.listdir(jack_label_path): # for each line
  text_file = os.path.join(jack_label_path,file_name)
  annotations = []
  with open(text_file) as f: # open text file
      for line in f: # for each line
        labels = line.split() 
        if (not labels[0].isdigit()):
          raise Exception(text_file, "label is not a digit (possible still a label not a number)") 
        if (int(labels[0])) == current_class_number:
          labels[0] = new_class_number
          newline = str(labels[0]) + " " + str(labels[1]) + " " + str(labels[2]) + " " + str(labels[3]) + " " + str(labels[4])
          line = line.replace(line, newline)
          annotations.append(line) # List of string for each line (replace first value with class number entered)
        else: 
          annotations.append(line) # List of string for each line (replace first value with class number entered)
      f.close()

  with open(text_file, "w") as outfile:
    for line in annotations:
        outfile.write(line)
    outfile.close()

C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train
